#Import

Reference: https://www.kaggle.com/phiitm/aspect-based-sentiment-analysis 

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!pip install -U spacy
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 2.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [28]:
from xml.etree import cElementTree as ET
import pandas as pd
import string
import spacy
import re

In [29]:
# Load tokenizer, tagger, parser, NER and word vectors
from spacy.lang.hi import Hindi
from spacy.lang.en import English 
from spacy.lang.en.stop_words import STOP_WORDS

In [30]:
# root_dir = "/content/drive/MyDrive/Topic 5/"
root_dir = "/content/drive/MyDrive/Acads/4-1/NLP/NLP Project/Topic 5/"
laptop_train_path = "Train/SemEval'14-ABSA-TrainData_v2 & AnnotationGuidelines/Laptop_Train_v2.xml"
restaurants_train_path = "Train/SemEval'14-ABSA-TrainData_v2 & AnnotationGuidelines/Restaurants_Train_v2.xml"
laptop_test_1_path = "Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/Laptops_Test_Data_PhaseA.xml"
laptop_test_2_path = "Test2/ABSA_TestData_PhaseB/Laptops_Test_Data_phaseB.xml"
restaurants_test_1_path = "Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/Restaurants_Test_Data_PhaseA.xml"
restaurants_test_2_path = "Test2/ABSA_TestData_PhaseB/Restaurants_Test_Data_phaseB.xml"

# Data Extraction & Cleaning

In [31]:
def clean_data(data, categorize = True):
    if not categorize:
        data = data.drop(['aspect_categories'], axis=1)
    return data.dropna()

def xml_to_df(path, clean = False, categorize = True, use_sentence_id_as_id = True):
    columns = ['sentence_id', 'text','aspect_terms','aspect_categories']
    if use_sentence_id_as_id:
        df = pd.DataFrame(columns = columns[1:])
    else:
        df = pd.DataFrame(columns = columns)
    
    tree = ET.parse(path)
    root = tree.getroot()

    for page in root.findall('sentence'):
        if use_sentence_id_as_id:
            temp = [page[0].text]
        else:
            temp = [page.attrib["id"], page[0].text]
        for i in range(1,len(page)):
            temp.append([x.attrib for x in page[i]])
        temp+=[None]*(3-len(page))
        if use_sentence_id_as_id:
            df.loc[int(page.attrib["id"])] = temp
        else:
            df.loc[len(df)] = temp
            

    if (clean):
        return clean_data(df, categorize)
    else:
        return df

In [32]:
laptop_train_data = xml_to_df(root_dir+laptop_train_path, clean = True, categorize=False)
laptop_train_data.head()

,text,aspect_terms
2339,I charge it at night and skip taking the cord ...,"[{'term': 'cord', 'polarity': 'neutral', 'from..."
1316,The tech guy then said the service center does...,"[{'term': 'service center', 'polarity': 'negat..."
2005,"it is of high quality, has a killer GUI, is ex...","[{'term': 'quality', 'polarity': 'positive', '..."
2789,Easy to start up and does not overheat as much...,"[{'term': 'start up', 'polarity': 'positive', ..."
76,"I even got my teenage son one, because of the ...","[{'term': 'features', 'polarity': 'positive', ..."


In [33]:
# Split into test and train
laptop_train_data_test = laptop_train_data.sample(frac = 0.1)
laptop_train_data = laptop_train_data.drop(laptop_train_data_test.index)

# Preprocessing

In [34]:
'''
data is a pandas Dataframe
'''

# def to_lowercase(data):
#     for ind in data.index:
#         for col in data.columns:
#             if (isinstance(data[col][ind], str)):
#                 data[col][ind] = data[col][ind].lower()
#             elif (isinstance(data[col][ind], list)):
#                 for i in range(len(data[col][ind])):
#                     for k in data[col][ind][i].keys():
#                         data[col][ind][i][k] = data[col][ind][i][k].lower()
#     return data

nlp = English()
nlp.add_pipe('sentencizer')

def tokenize(data):
    for i in data.index:
        text = nlp(data['text'][i])
        token_list = []
        for token in text:
            token_list.append(token.text)
        data['text'][i] = token_list

# def stop_word_removal(data, stop_words = ["I", "we", "they", "of", "at", "between", "is", "am", "are", "as", "a", "an", "the", "me", "to", "in", "it", "that", "had", "on", "for", "were", "was"]):
#     # Create list of word tokens after removing stopwords
#     for i in data.index:
#         filtered_tokens =[]
#         for word in data['text'][i]:
#             lexeme = nlp.vocab[word]
#             if word not in stop_words and (lexeme.is_punct == False or word == ','):
#                 filtered_tokens.append(word)
#         data['text'][i] = filtered_tokens

# def splitByAspectTerms(data):
#     split_sentences[]
    
#     for ind in data.index:
#         for aspect_term in data['aspect_terms'][ind]:
#             df

In [35]:
tokenize(laptop_train_data)
laptop_train_data.head()

,text,aspect_terms
2339,"[I, charge, it, at, night, and, skip, taking, ...","[{'term': 'cord', 'polarity': 'neutral', 'from..."
1316,"[The, tech, guy, then, said, the, service, cen...","[{'term': 'service center', 'polarity': 'negat..."
2005,"[it, is, of, high, quality, ,, has, a, killer,...","[{'term': 'quality', 'polarity': 'positive', '..."
2789,"[Easy, to, start, up, and, does, not, overheat...","[{'term': 'start up', 'polarity': 'positive', ..."
76,"[I, even, got, my, teenage, son, one, ,, becau...","[{'term': 'features', 'polarity': 'positive', ..."


In [36]:
pos_tagger = spacy.load("en_core_web_sm")
test = "The display on this computer is the best I've seen in a very long time, the battery life is very long and very convenient."
doc = pos_tagger(test)
spacy.displacy.render(doc,style='dep',jupyter=True)

# Generate Aspect Terms

In [37]:
def find_aspect_terms(data):
    aspect_terms = []
    
    columns = ["base_noun", "gen_aspect_terms", "sentiment_words"]
    aspect_terms_df = pd.DataFrame(columns = columns)
    for i in data.index:
        line = ' '.join(data['text'][i])

        # pos_tagger = spacy.load('en_core_web_lg', parse=True, tag=True, entity=True)
        pos_tagger = spacy.load("en_core_web_sm")

        doc = pos_tagger(line)

        # spacy.displacy.render(doc,style='dep',jupyter=True)
        
        str1=''
        str2=''
        for token in doc:
            row = {"base_noun":'', "gen_aspect_terms":'', "sentiment_words":[]}
            if token.pos_ is 'NOUN':
                row["base_noun"] = token.text
                potential_aspect_term = ""
                for j in token.lefts:
                    if j.dep_ == 'compound':
                        potential_aspect_term += (j.text + " ")
                        aspect_terms_df = (aspect_terms_df.loc[aspect_terms_df['base_noun'] != j.text]) 
            
                potential_aspect_term += token.text

                row["gen_aspect_terms"] = potential_aspect_term
                        
                for j in token.lefts:
                    if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                        row["sentiment_words"].append(j.text)
                        for k in j.lefts:
                            if k.dep_ is 'advmod' or k.dep_ is 'neg': #secondary condition to get adjective of adjectives
                                row["sentiment_words"].append(k.text+' '+j.text)
                
                aspect_terms_df = aspect_terms_df.append(row, ignore_index = True)

            if token.pos_ is 'VERB':
                for j in token.lefts:
                    if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                        aspect_terms_df.loc[len(aspect_terms_df)] = [token.text, token.text, [j.text]]
                    if j.dep_ is 'neg' and j.pos_ is 'ADV':
                        aspect_terms_df.loc[len(aspect_terms_df)] = [token.text, token.text, [j.text]]
                for j in token.rights:
                    if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                        aspect_terms_df.loc[len(aspect_terms_df)] = [token.text, token.text, [j.text]]

            # if token.pos_ is 'ADJ':
            #     for j,h in zip(token.rights,token.lefts):
            #         if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
            #             for k in j.lefts:
            #                 if k.dep_ is 'aux':
            #                     xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
            #         elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
            #             if k.dep_ is 'aux':
            #                     neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)   
        aspect_terms_df = (aspect_terms_df.drop(columns = ["base_noun"]))
        aspect_terms_row = []
        for ind in aspect_terms_df.index:
            aspect_terms_row.append({aspect_terms_df["gen_aspect_terms"][ind]:aspect_terms_df["sentiment_words"][ind]})

        aspect_terms.append([i,aspect_terms_row])
        
    aspect_terms = pd.DataFrame(aspect_terms, columns = ["sentence_id", "gen_aspect_terms"])
    aspect_terms = aspect_terms.set_index("sentence_id")
    return aspect_terms

In [38]:
laptop_train_data_aspect_terms = find_aspect_terms(laptop_train_data)
laptop_train_data_aspect_terms

ValueError: ignored

In [39]:
stop_words = ['so','be','are','just','get','were','When','when','again','where','how','has','Here','here','now','see','why']

In [40]:
nlp = spacy.load('en_core_web_sm')

In [41]:
def find_aspect_terms(data, nlp):
    aspect_terms = []
    comp_terms = []
    easpect_terms = []
    ecomp_terms = []
    enemy = []
    for i in data.index:
        amod_pairs = []
        advmod_pairs = []
        compound_pairs = []
        xcomp_pairs = []
        neg_pairs = []
        
        line = ' '.join(x for x in data['text'][i] if x not in stop_words)
        doc = nlp(line)
        str1=''
        str2=''
        for token in doc:
            if token.pos_ is 'NOUN':
                for j in token.lefts:
                    if j.dep_ == 'compound':
                        compound_pairs.append((j.text+' '+token.text,token.text))
                    if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                        str1 = j.text+' '+token.text
                        amod_pairs.append(j.text+' '+token.text)
                        for k in j.lefts:
                            if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                str2 = k.text+' '+j.text+' '+token.text
                                amod_pairs.append(k.text+' '+j.text+' '+token.text)
                        mtch = re.search(re.escape(str1),re.escape(str2))
                        if mtch is not None:
                            amod_pairs.remove(str1)
            if token.pos_ is 'VERB':
                for j in token.lefts:
                    if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                        advmod_pairs.append(j.text+' '+token.text)
                    if j.dep_ is 'neg' and j.pos_ is 'ADV':
                        neg_pairs.append(j.text+' '+token.text)
                for j in token.rights:
                    if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                        advmod_pairs.append(token.text+' '+j.text)
            if token.pos_ is 'ADJ':
                for j,h in zip(token.rights,token.lefts):
                    if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                        for k in j.lefts:
                            if k.dep_ is 'aux':
                                xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                    elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                        if k.dep_ is 'aux':
                                neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
            pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
            for i in range(len(pairs)):
                if len(compound_pairs)!=0:
                    for comp in compound_pairs:
                        mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                        if mtch is not None:
                            pairs[i] = pairs[i].replace(mtch.group(),comp[0])
                
        aspect_terms.append(pairs)
        comp_terms.append(compound_pairs)

    return aspect_terms, comp_terms

In [42]:
laptop_train_data_aspect_terms_2, laptop_train_data_comp_terms_2 = find_aspect_terms(laptop_train_data, nlp)
print(laptop_train_data_aspect_terms_2)
print(laptop_train_data_comp_terms_2)

[['good battery life'], ['retail shop', 'then said'], ['good applications', 'high quality', 'very good applications'], ['other laptops', 'overheat much'], ['even got', 'teenage son'], ['many features', 'great features', 'Great laptop'], ['hard drive', 'usually does', 'next day', 'steady drive'], ['same screen', 'blue screen', 'took back', 'thing- screen'], ['clever con', 'never know', 'very clever con', 'soft rubber enclosure'], ['external mouse', 'large tracking area', 'However make', 'multi touch'], ['works together', 'entire suite'], [], [], ['barely use', 'stay properly'], ['finally had'], [], ['pure pleasure talk', 'take forever'], ['really helps', 'also got'], ['crashes completely', 'occasionally crashes'], [], ['preloaded software'], ['best thing', 'newer features'], ['setting directly', 'numerous attempts'], [], [], ['sent back', 'sent twice'], [], [], [], [], [], [], [], ['Nightly defrags'], [], [], [], [], [], ['personal files', 'mainly use'], [], ['working all', 'work proper

In [ ]:
def aspect_term_performance(train_data, generated_aspect_terms):
    true_positives = 0
    false_positives = 0
    # false_negatives = 0

    for i in train_data.index:
        # List of actual aspect_terms
        actual_aspect_terms_i = [x['term'] for x in train_data["aspect_terms"][i]]
        generated_aspect_terms_at_i = [list(x.keys())[0] for x in generated_aspect_terms["gen_aspect_terms"][i]]

        for terms in generated_aspect_terms_at_i:
            found = False
            for actual_terms in actual_aspect_terms_i:
                if re.search(re.escape(terms), re.escape(actual_terms)):
                    true_positives += len(terms.split())
                    found = True
                    break
            if not found:
                false_positives += len(terms.split())

    ratios = {"Precision": true_positives / (true_positives + false_positives)}
    return ratios

In [ ]:
laptop_train_data_ratios = aspect_term_performance(laptop_train_data, laptop_train_data_aspect_terms)
print(laptop_train_data_ratios)

{'Precision': 0.23317906728373086}


#Train the model
Make a frequency dictionary for all the Sentiment Words

In [ ]:
def train(train_data, generated_aspect_terms):
    '''
    Both are dataframes
    '''
    # Do preprocessing if needed
    sentiment_words_frequency = {}
    for i in train_data.index:
        generated_aspect_terms_at_i = generated_aspect_terms.loc[i]
        # print(generated_aspect_terms_at_i)
        for x in train_data['aspect_terms'][i]:
            aspect_term = x['term']
            if x['polarity'] == 'positive':
                polarity = 1
            elif x['polarity'] == 'negative':
                polarity = -1
            else:
                polarity = 0
            
            for x in generated_aspect_terms_at_i["gen_aspect_terms"]:
                aspect_term = list(x.keys())[0]
                sentiment_words = x[aspect_term]
                for sentiment_word in sentiment_words:
                    # print(sentiment_word)
                    if sentiment_word in list(sentiment_words_frequency.keys()): 
                        sentiment_words_frequency[sentiment_word]['frequency'] += 1
                        sentiment_words_frequency[sentiment_word]['polarity'] += (polarity)
                    else:
                        sentiment_words_frequency[sentiment_word] = {'frequency':1, 'polarity':polarity}

    for key in list(sentiment_words_frequency.keys()):
        sentiment_words_frequency[key] = (sentiment_words_frequency[key]['polarity']/sentiment_words_frequency[key]['frequency'])

    return sentiment_words_frequency

In [ ]:
laptop_sentiment_words_frequency_train = train(laptop_train_data, laptop_train_data_aspect_terms)
print(len(laptop_sentiment_words_frequency_train))
print(laptop_sentiment_words_frequency_train)

599
{'good': 0.7674418604651163, 'then': -0.6206896551724138, 'retail': -0.75, 'high': 0.5, 'very good': 0.8888888888888888, 'other': 0.28125, 'even': -0.23076923076923078, 'teenage': 1.0, 'Great': 1.0, 'many': 0.2653061224489796, 'great': 0.7474747474747475, 'next': -0.3, 'dark': -0.16666666666666666, 'usually': -0.375, 'back': -0.6730769230769231, 'same': 0.04, 'blue': 0.25, 'soft': 0.0, 'so': -0.2, 'never': -0.16216216216216217, 'home': -0.3333333333333333, 'clever': 0.0, 'very clever': 0.0, 'multi': 0.5833333333333334, '-': 0.3125, 'touch': 0.5833333333333334, 'large': 0.5, 'However': 0.36363636363636365, 'external': -0.13043478260869565, 'entire': -0.5, 'together': 0.0, 'barely': 0.3333333333333333, 'properly': -0.4, 'When': -0.38461538461538464, 'finally': -0.2222222222222222, 'just': 0.0684931506849315, 'pure': 1.0, 'painless': 1.0, 'forever': 0.0, 'also': 0.22727272727272727, 'really': 0.0, 'occasionally': -1.0, 'completely': -0.4, 'once': 0.18181818181818182, 'ago': -0.75, 'pe

# Test the Model

In [ ]:
def predict(test_data, sentiment_words_frequency):
    generated_aspect_terms = find_aspect_terms(test_data)
    all_unk = 0
    for i in generated_aspect_terms.index:
        for x in generated_aspect_terms["gen_aspect_terms"][i]:
            aspect_term = list(x.keys())[0]
            sentiment_words = x[aspect_term]
            unk = 0
            polarity = 0
            for word in sentiment_words:
                if word not in list(sentiment_words_frequency.keys()):
                    unk += 1
                else:
                    polarity += sentiment_words_frequency[word]
            if (len(sentiment_words) - unk != 0):
                x[aspect_term] = polarity/(len(sentiment_words)-unk)
            else:
                x[aspect_term] = 0
            all_unk += unk
    data = pd.concat([test_data, generated_aspect_terms], axis=1, join="inner")
    return data, all_unk

def classify(predictions):
    classes = []
    for i in range(predictions.shape[0]):
        terms_in_classes = []
        aspect_terms_at_i = predictions['gen_aspect_terms'].iloc[i] 
        for terms in aspect_terms_at_i:
            aspect_term = list(terms.keys())[0]
            polarity = terms[aspect_term]
            
            if (polarity < 0):
                sentiment = "negative"
            elif (polarity > 0):
                sentiment = "positive"
            else:
                sentiment = "neutral"

            terms_in_classes.append({aspect_term:sentiment})
        classes.append(terms_in_classes)
    predictions['classified_aspect_terms'] = classes

In [ ]:
laptop_test_1_data = xml_to_df(root_dir+laptop_test_1_path, clean = False, categorize = False, use_sentence_id_as_id = False)
tokenize(laptop_test_1_data)
laptop_test_1_data.head()

,sentence_id,text,aspect_terms,aspect_categories
0,892:1,"[Boot, time, is, super, fast, ,, around, anywh...",None,None
1,1144:1,"[tech, support, would, not, fix, the, problem,...",None,None
2,805:2,"[but, in, resume, this, computer, rocks, !]",None,None
3,359:1,"[Set, up, was, easy, .]",None,None
4,562:1,"[Did, not, enjoy, the, new, Windows, 8, and, t...",None,None


In [ ]:
laptop_test_1_predictions, laptop_test_1_predictions_unk_count = predict(laptop_test_1_data[40:50],laptop_sentiment_words_frequency_train)
classify(laptop_test_1_predictions)
print("Unknown Sentiment Words:", laptop_test_1_predictions_unk_count)
laptop_test_1_predictions.head()

Unknown Sentiment Words: 0


,sentence_id,text,aspect_terms,aspect_categories,gen_aspect_terms,classified_aspect_terms
40,470:1,"[This, is, why, I, purchased, a, BRAND, NEW, L...",None,None,"[{'purchased': -0.7142857142857143}, {'place':...","[{'purchased': 'negative'}, {'place': 'positiv..."
41,499:1,"[It, has, so, much, more, speed, and, the, scr...",None,None,"[{'speed': 0.8285714285714285}, {'screen': 0}]","[{'speed': 'positive'}, {'screen': 'neutral'}]"
42,457:1,"[As, for, the, laptop, ,, this, is, our, 3rd, ...",None,None,"[{'laptop': 0}, {'Apple computer': -1.0}, {'ye...","[{'laptop': 'neutral'}, {'Apple computer': 'ne..."
43,636:1,"[Everything, I, wanted, and, everything, I, ne...",None,None,[{'price': 0}],[{'price': 'neutral'}]
44,48:1,"[It, 's, not, inexpensive, but, the, Hardware,...",None,None,"[{'Hardware performance': 0}, {'computer': 0}]","[{'Hardware performance': 'neutral'}, {'comput..."


In [ ]:
laptop_test_1_predictions.to_csv(path_or_buf = root_dir+"Test1/ABSA_TestData_PhaseA/ABSA_TestData_PhaseA/"+"Spacy_Pred.csv")

# Performance of Sentiment Classification

In [ ]:
# Split data and do this
def classification_performance(test_data_predicted, test_data_with_sentiments):
    matches = 0
    non_matches = 0
    for i in test_data_predicted.index:
        aspect_terms_i = test_data_predicted["classified_aspect_terms"][i]
        actual_aspect_terms_i = test_data_with_sentiments["aspect_terms"][i]
        for term in aspect_terms_i:
            aspect_term = list(term.keys())[0]
            polarity = term[aspect_term]
            for actual_term in actual_aspect_terms_i:
                if (aspect_term == actual_term['term']):
                    if (polarity == actual_term['polarity']):
                        matches+=1
                    else:
                        non_matches+=1

    return {"Matches":matches, "Non Matches":non_matches, "Ratio":matches/(matches+non_matches+0.001)}

In [ ]:
tokenize(laptop_train_data_test)
laptop_train_data_test_predictions, laptop_train_data_test_predictions_unk_count  =  predict(laptop_train_data_test, laptop_sentiment_words_frequency_train)
classify(laptop_train_data_test_predictions)
print("Unknown Sentiment Words:", laptop_train_data_test_predictions_unk_count)
laptop_train_data_test_predictions.head()

Unknown Sentiment Words: 38


,text,aspect_terms,gen_aspect_terms,classified_aspect_terms
300,"[I, run, Dreamweaver, ,, Final, Cut, Pro, 7, ,...","[{'term': 'applications', 'polarity': 'neutral...","[{'run': -0.8888888888888888}, {'applications'...","[{'run': 'negative'}, {'applications': 'positi..."
2482,"[We, love, the, size, of, the, screen, ,, alth...","[{'term': 'size of the screen', 'polarity': 'p...","[{'size': 0}, {'screen': 0}, {'is': -0.3478260...","[{'size': 'neutral'}, {'screen': 'neutral'}, {..."
2727,"[Tried, to, make, a, recovey, disk, would, nt,...","[{'term': 'recovey disk', 'polarity': 'negativ...","[{'recovey disk': 0}, {'recovery disk': 0.0357...","[{'recovey disk': 'neutral'}, {'recovery disk'..."
2097,"[For, me, I, was, lucky, and, a, local, store,...","[{'term': 'price', 'polarity': 'positive', 'fr...","[{'store': 0}, {'price': 0}]","[{'store': 'neutral'}, {'price': 'neutral'}]"
2707,"[Would, like, more, trendy, ,, high, tech, fea...","[{'term': 'features', 'polarity': 'negative', ...",[{'tech features': 0}],[{'tech features': 'neutral'}]


In [ ]:
laptop_train_data_test_ratios = classification_performance(laptop_train_data_test_predictions, laptop_train_data_test)
print(laptop_train_data_test_ratios)

{'Matches': 43, 'Non Matches': 119, 'Ratio': 0.2654304603058006}
